In [6]:
import os
import pandas as pd
import numpy as np

# Coordinate file (contain a column with station IDs)
COORD_FILE = "410_data/14_Station_Coordinates__Updated_.csv"
# Folder that contains all NCEI normals CSV files
DATA_DIR   = "410_data"
# Output file: stations × 365 matrix
OUT_FILE   = "410_data/matrix_14stations_365.csv"


coords = pd.read_csv(COORD_FILE)

# Here we assume the coordinate table has a column named "file"
# whose values look like USC00111577, USC00300183, ...
station_ids = coords["file"].astype(str).tolist()


def read_station_series(path):
    """
    From a single NCEI Daily Climate Normals file, extract a 365-length
    Tmean (DLY-TAVG-NORMAL) series indexed by day-of-year (1..365).

    The function:
      * finds a Tmean/TAVG normals column (case-insensitive),
      * builds a day-of-year index (1..366),
      * interpolates missing values,
      * removes DOY 366 (leap day) and returns only 1..365.
    """
    df = pd.read_csv(path, low_memory=False)

    #Find the Tmean column
    t_candidates = []
    for c in df.columns:
        cname = str(c).lower()
        if (
            "dly-tavg-normal" in cname
            or ("tavg" in cname and "normal" in cname)
            or ("tmean" in cname and "normal" in cname)
        ):
            t_candidates.append(c)

    if not t_candidates:
        raise ValueError(f"No Tmean-like column found in {os.path.basename(path)}")

    tcol = t_candidates[0]

    # Build day-of-year: 1 - 366
    cols_lower = {c: str(c).lower() for c in df.columns}
    doy_col = None

    #If the file already has a DOY/dayofyear/julian/yearday column, use it
    for c in df.columns:
        cl = cols_lower[c]
        if cl in ["doy", "dayofyear", "julian", "yearday"]:
            doy_col = c
            break

    if doy_col is not None:
        sub = df[[doy_col, tcol]].copy()
        sub[doy_col] = pd.to_numeric(sub[doy_col], errors="coerce")
        sub[tcol]    = pd.to_numeric(sub[tcol],    errors="coerce")
        sub = sub.dropna(subset=[doy_col, tcol])
        sub = sub[(sub[doy_col] >= 1) & (sub[doy_col] <= 366)]
        # If there are duplicate days, average them
        sub = sub.groupby(doy_col)[tcol].mean()
        series_full = pd.Series(index=range(1, 367), dtype=float)
        series_full.loc[sub.index.astype(int)] = sub.values

    # Otherwise, if there are month+day columns, compute DOY using year 2000
    elif {"month", "day"}.issubset(set(df.columns)):
        tmp = df[["month", "day", tcol]].copy()
        tmp["month"] = pd.to_numeric(tmp["month"], errors="coerce")
        tmp["day"]   = pd.to_numeric(tmp["day"],   errors="coerce")
        tmp[tcol]    = pd.to_numeric(tmp[tcol],    errors="coerce")
        tmp = tmp.dropna(subset=["month", "day", tcol])

        # Year 2000 is a leap year, so it can represent Feb 29 (DOY = 60)
        tmp["doy"] = pd.to_datetime(
            {
                "year": 2000,
                "month": tmp["month"].astype(int),
                "day":   tmp["day"].astype(int),
            },
            errors="coerce",
        ).dt.dayofyear

        tmp = tmp.dropna(subset=["doy"])
        tmp["doy"] = tmp["doy"].astype(int)
        tmp = tmp[(tmp["doy"] >= 1) & (tmp["doy"] <= 366)]

        sub = tmp.groupby("doy")[tcol].mean()
        series_full = pd.Series(index=range(1, 367), dtype=float)
        series_full.loc[sub.index] = sub.values

    #Fallback: assume the file is already one row per day in order
    else:
        s = pd.to_numeric(df[tcol], errors="coerce")
        series_full = pd.Series(index=range(1, 367), dtype=float)
        n = min(len(s), 366)
        series_full.iloc[:n] = s.iloc[:n].values

    #Interpolate and drop DOY 366 to get 365 days 
    series_full = series_full.interpolate(limit_direction="both")

    # Keep only DOY 1..365
    series365 = series_full.iloc[:365]

    # Safety check: if there are still NaNs, interpolate again
    if series365.isna().any():
        series365 = series365.interpolate(limit_direction="both")

    return series365


# ==============================
# Main loop: build stations × 365 matrix
# ==============================

rows = []
good_ids = []

for sid in station_ids:
    f = os.path.join(DATA_DIR, f"{sid}.csv")
    if not os.path.exists(f):
        print("Missing file for station:", sid)
        continue

    try:
        series365 = read_station_series(f)
        if len(series365) != 365:
            raise ValueError(f"Got length {len(series365)} for {sid}")
        rows.append(series365.values)
        good_ids.append(sid)
    except Exception as e:
        print("Skip", sid, "->", e)

if rows:
    # Stack into a matrix: stations × 365
    X = np.vstack(rows)
    days = list(range(1, 366))

    matrix_df = pd.DataFrame(X, index=good_ids, columns=days)

    # Final QC: if any NaNs remain, interpolate along each row
    if matrix_df.isna().any().any():
        matrix_df = matrix_df.apply(
            lambda r: pd.Series(r).interpolate(limit_direction="both"), axis=1
        )
        matrix_df.index = good_ids
        matrix_df.columns = days

    matrix_df.to_csv(OUT_FILE)
    print("Saved:", OUT_FILE, "shape =", matrix_df.shape)
else:
    print("No station series were built; please check DATA_DIR and column names.")


Saved: 410_data/matrix_14stations_365.csv shape = (14, 365)
